In [1]:
import torch
import torch.nn as nn
from torchvision import datasets,transforms, models
import torch.optim as optim
from models import Net_purchase,Net_texas
from train_process import Train_shadowmodel, Train_targetmodel, Load_shadowmodel, Load_targetmodel
import os
import argparse
import numpy as np
from my_dataloader import dataloader
from train_eval import train, eval_model
import matplotlib.pyplot as plt  
from tqdm import tqdm
from Compute_score import *
import random

In [2]:
def setup_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
setup_seed(1)

torch.cuda.is_available()

True

In [3]:
dataset = 'purchase'#The dataset of choice between"mnist"  ，"fashionmnist", "emnist","cifar" ,"cifar100","purchase" and "texas".
c_num = 100

Net = Net_purchase
loss = nn.CrossEntropyLoss()
batch_size = 100
target_in_loader,refen_a_loader,refen_b_loader,shadow_in_loader,shadow_out_loader,testloader = dataloader(dataset, batch_size)

In [6]:
# selecte 20 models as reference models
ref_model = []
random_vector = np.random.choice(100, 20, replace=False).tolist()
for i in range(20):
    path = "./ref_"+ str(dataset)+"/ref_"+ str(random_vector[i]) + "_"+ str(dataset) + ".pth"
    model = Net()
    model.load_state_dict(torch.load(path)) 
    ref_model.append(model)

In [8]:
path =  "./target_shadow/target_" + "_"+ str(dataset) + ".pth"
target_net = Net()
target_net.load_state_dict(torch.load(path)) 
path =  "./target_shadow/shadow_" + "_"+ str(dataset) + ".pth"
shadow_net = Net()
shadow_net.load_state_dict(torch.load(path)) 

<All keys matched successfully>

In [9]:
lossf = nn.CrossEntropyLoss(reduce=False)
with torch.no_grad():
    shadow0_in, shadow1_in = Com_Loss(shadow_net, ref_model, shadow_in_loader, lossf)
    shadow0_out, shadow1_out = Com_Loss(shadow_net, ref_model, shadow_out_loader, lossf)
    Score0_in,Score1_in = Com_Loss(target_net, ref_model, target_in_loader, lossf)
    Score0_out,Score1_out = Com_Loss(target_net, ref_model, testloader, lossf)

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
100%|██████████| 20/20 [00:07<00:00,  2.53it/s]


In [10]:
mu_list = [0.9,0.92,0.94,0.96,0.98]
for i in range(len(mu_list)):
    mu = mu_list[i]
    t0,t1 = Twostage_create(shadow0_in,shadow1_in,shadow0_out,shadow1_out,mu)
    TP,FP,TN,FN = Twostage_attack_eval(Score0_in,Score1_in,Score0_out,Score1_out,t0,t1)
    if TP + FP != 0:
        print('Two-stage:mu = %f,Pr = %f, Recall/TPR = %f,FPR = %f, TP = %d'%(mu,TP/(TP+FP),TP/(TP+FN),FP/(TN+FP),TP))
    else:
        print('Two-stage:mu = %f,TP+FP = %d,TP = %d'%(mu,TP+FP,TP))
    t,flag = Onestage_create(shadow1_in,shadow1_out,mu)
    TP,FP,TN,FN = Onestage_attack_eval(Score1_in,Score1_out,t)
    if TP + FP != 0:
        print('C-Loss:mu = %f,Pr = %f, Recall/TPR = %f,FPR = %f, TP = %d'%(mu,TP/(TP+FP),TP/(TP+FN),FP/(TN+FP),TP))
    else:
        print('C-Loss:mu = %f,flag = %d,TP+FP = %d,TP = %d'%(mu,flag,TP+FP,TP))

Two-stage:mu = 0.900000,Pr = 0.918182, Recall/TPR = 0.035350,FPR = 0.003150, TP = 707
C-Loss:mu = 0.900000,Pr = 0.928571, Recall/TPR = 0.000650,FPR = 0.000050, TP = 13
Two-stage:mu = 0.920000,Pr = 0.920502, Recall/TPR = 0.011000,FPR = 0.000950, TP = 220
C-Loss:mu = 0.920000,Pr = 0.916667, Recall/TPR = 0.000550,FPR = 0.000050, TP = 11
Two-stage:mu = 0.940000,Pr = 0.917197, Recall/TPR = 0.007200,FPR = 0.000650, TP = 144
C-Loss:mu = 0.940000,Pr = 0.916667, Recall/TPR = 0.000550,FPR = 0.000050, TP = 11
Two-stage:mu = 0.960000,Pr = 0.942857, Recall/TPR = 0.003300,FPR = 0.000200, TP = 66
C-Loss:mu = 0.960000,Pr = 0.916667, Recall/TPR = 0.000550,FPR = 0.000050, TP = 11
Two-stage:mu = 0.980000,Pr = 1.000000, Recall/TPR = 0.001050,FPR = 0.000000, TP = 21
C-Loss:mu = 0.980000,Pr = 0.916667, Recall/TPR = 0.000550,FPR = 0.000050, TP = 11
